# Fast testing

In [1]:
import sys
sys.path.insert(0, "../")
from AC_modules.BatchedA2C import FullSpaceA2C_v2
from SC_Utils.game_utils import FullObsProcesser
from SC_Utils.train_v5 import *
import AC_modules.Networks as net
import torch

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Environment parameters
RESOLUTION = 32
game_params = dict(feature_screen=RESOLUTION, feature_minimap=RESOLUTION, action_space="FEATURES") 
game_names = {1:'MoveToBeacon',
              2:'CollectMineralShards',
              3:'DefeatRoaches',
              4:'FindAndDefeatZerglings',
              5:'DefeatZerglingsAndBanelings',
              6:'CollectMineralsAndGas',
              7:'BuildMarines'
              }
map_name = game_names[1]
obs_proc_params = {'select_all':True}
action_names = ['select_army','Move_screen','select_point','Attack_screen']
action_names = ['move_camera', 'select_point', 'select_rect', 
                'select_idle_worker', 'select_army', 'Attack_screen',
                'Attack_minimap', 'Build_Barracks_screen', 'Build_CommandCenter_screen',
                'Build_Refinery_screen', 'Build_SupplyDepot_screen',
                'Harvest_Gather_SCV_screen', 'Harvest_Return_SCV_quick',
                'HoldPosition_quick', 'Move_screen', 'Move_minimap',
                'Rally_Workers_screen', 'Rally_Workers_minimap',
                'Train_Marine_quick', 'Train_SCV_quick']

In [3]:
env = init_game(game_params, map_name)
op = FullObsProcesser(**obs_proc_params)
screen_channels, minimap_channels, in_player = op.get_n_channels()
in_channels = screen_channels + minimap_channels 

In [4]:
spatial_model = net.FullyConvPlayerAndSpatial
nonspatial_model = net.FullyConvNonSpatial
# Internal features, passed inside a dictionary
conv_channels = 32
player_features = 16
# Exposed features, passed outside of a dictionary
n_channels = 48
n_features = 256
spatial_dict = {"in_channels":in_channels, 'in_player':in_player, 
                'conv_channels':conv_channels, 'player_features':player_features}
nonspatial_dict = {'resolution':RESOLUTION, 'kernel_size':3, 'stride':2, 'n_channels':n_channels}

In [5]:
HPs = dict(gamma=0.99, n_steps=20, H=1e-2, 
           spatial_model=spatial_model, nonspatial_model=nonspatial_model,
           n_features=n_features, n_channels=n_channels, action_names=action_names,
           spatial_dict=spatial_dict, nonspatial_dict=nonspatial_dict)

if torch.cuda.is_available():
    HPs['device'] = 'cuda'
else:
    HPs['device'] = 'cpu'
    
print("Using device "+HPs['device'])

lr = 7e-4

Using device cpu


In [6]:
agent = FullSpaceA2C_v2(env=env, **HPs)
env.close()

In [7]:
unroll_length = 60

train_dict = dict(n_train_processes = 2,
                  max_train_steps = unroll_length*1000,#60000,
                  unroll_length = unroll_length,
                  test_interval = unroll_length*1,#10
                  inspection_interval = unroll_length*1#10
                  )

In [8]:
%%time
results = train_batched_A2C(agent, game_params, map_name, lr, 
                            obs_proc_params=obs_proc_params, **train_dict)

Process ID:  ZZBQ


Process Process-1:
Process Process-2:
Traceback (most recent call last):
  File "/home/nicola/anaconda3/envs/torch/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/nicola/anaconda3/envs/torch/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "../SC_Utils/train_v5.py", line 49, in worker
    cmd, data = worker_end.recv()
  File "/home/nicola/anaconda3/envs/torch/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/nicola/anaconda3/envs/torch/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/nicola/anaconda3/envs/torch/lib/python3.7/multiprocessing/connection.py", line 250, in recv
    buf = self._recv_bytes()
  File "../SC_Utils/train_v5.py", line 49, in worker
    cmd, data = worker_end.recv()
  File "/home/nicola/anaconda3/envs/torch

Traceback (most recent call last):
  File "/home/nicola/anaconda3/envs/torch/lib/python3.7/site-packages/IPython/core/magics/execution.py", line 1310, in time
    exec(code, glob, local_ns)
  File "<timed exec>", line 2, in <module>
  File "../SC_Utils/train_v5.py", line 224, in train_batched_A2C
    score.append(avg_score)
  File "../SC_Utils/train_v5.py", line 259, in test
    obs = test_env.step(a)
  File "../AC_modules/BatchedA2C.py", line 481, in step
    a = a.detach().cpu().numpy()
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/nicola/anaconda3/envs/torch/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2040, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/nicola/anacon

KeyboardInterrupt: 

# Including player information in the state

In [ ]:
env = init_game(game_params, map_name)

In [ ]:
obs = env.reset()

In [ ]:
player = obs[0].observation['player']
player

In [ ]:
list(player._index_names[0].keys())

Info that I will exclude:
- player_id is always 1
- warp_gate_count and larva_count are useless during minigames

Preprocessing of the rest:
- apply a log2 to minerals and vespene, because being resources they can grow a lot in value
- cast to float the rest

In [ ]:
class PlayerProcesser():
    def __init__(self):
        self.useful_indexes = np.arange(1,9)
    
    def get_player_info(self, obs):
        player = obs[0].observation['player'].astype(float)
        x = player[self.useful_indexes]
        f =  lambda x: np.log2(x) if x != 0 else x
        x['minerals'] = f(x['minerals'])
        x['vespene'] = f(x['vespene'])
        return np.array(x), np.array(list(x._index_names[0].keys()))

In [ ]:
pp = PlayerProcesser()
pp.get_player_info(obs)

In [10]:
env.close()

Merging this new feature in the old ObsProcesser class...

In [11]:
class FullObsProcesser(ObsProcesser):
    def __init__(self, screen_names=[], minimap_names=[], select_all=False):
        super().__init__(screen_names, minimap_names, select_all)
        self.useful_indexes = np.arange(1,9)
        
    def get_state(self, obs):
        feature_screen = obs[0].observation['feature_screen']
        feature_minimap = obs[0].observation['feature_minimap']
        player_info = obs[0].observation['player'].astype(float)
        
        screen_layers, screen_names = self._process_screen_features(feature_screen)
        minimap_layers, minimap_names = self._process_minimap_features(feature_minimap)
        player_features, player_names = self._process_player_features(player_info)
        state = {'screen_layers':screen_layers, 'minimap_layers':minimap_layers, 'player_features':player_features}
        names = {'screen_names':screen_names, 'minimap_names':minimap_names, 'player_names':player_names}
        return state, names
    
    def _process_player_features(self, player):
        x = player[self.useful_indexes]
        f =  lambda x: np.log2(x) if x != 0 else x
        x['minerals'] = f(x['minerals'])
        x['vespene'] = f(x['vespene'])
        return np.array(x), np.array(list(x._index_names[0].keys()))
    
    def get_n_channels(self):
        screen_channels, minimap_channels = super().get_n_channels()
        player_channels = len(self.useful_indexes)
        return screen_channels, minimap_channels, player_channels

In [12]:
fop = FullObsProcesser(select_all=True)
state, names = fop.get_state(obs)
fop.get_n_channels()

(26, 12, 8)

In [14]:
state

{'screen_layers': array([[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],
 
        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 1., 1., ..., 1., 1., 0.],
         [0., 1., 1., ..., 1., 1., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],
 
        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],
 
        ...,
 
        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., .

Following the workflow of the algorithm, now we have to modify this function
```python
def merge_screen_and_minimap(state_dict):
    screen = state_dict['screen_layers']
    minimap = state_dict['minimap_layers']
    if len(minimap) > 0:
        try:
            assert screen.shape[-2:] == minimap.shape[-2:], 'different resolutions'
        except:
            print("Shape mismatch between screen and minimap. They must have the same resolution.")
            print("Screen resolution: ", screen.shape[-2:])
            print("Minimap resolution: ", minimap.shape[-2:])

        state = np.concatenate([screen, minimap])
    elif len(minimap)==0 and len(screen) >0:
        state = screen
    else:
        raise Exception("Both screen and minimap seem to have 0 layers.")
    return state              
```

In [13]:
def merge_screen_and_minimap(state_dict):
    """
    Returns a tuple (state, player), where
    state = (screen+minimap channels, res, res) # no batch dim
    player = (player_features,) # no batch dim
    """
    screen = state_dict['screen_layers']
    minimap = state_dict['minimap_layers']
    player = state_dict['player_features']
    if len(minimap) > 0:
        try:
            assert screen.shape[-2:] == minimap.shape[-2:], 'different resolutions'
        except:
            print("Shape mismatch between screen and minimap. They must have the same resolution.")
            print("Screen resolution: ", screen.shape[-2:])
            print("Minimap resolution: ", minimap.shape[-2:])

        state = np.concatenate([screen, minimap])
    elif len(minimap)==0 and len(screen) >0:
        state = screen
    else:
        raise Exception("Both screen and minimap seem to have 0 layers.")
    return state, player              

Let's check that the workers can send the tuple of the state correctly and then using 2 parallel envs let's batch the states together the new state representations keeping state and player separated.

In [15]:
n_train_processes = 2
envs = ParallelEnv(n_train_processes, game_params, map_name, obs_proc_params, action_dict)

In [16]:
state, action_mask = envs.reset()

In [17]:
state

{'spatial': array([[[[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],
 
         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 1., 1., ..., 1., 1., 0.],
          [0., 1., 1., ..., 1., 1., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],
 
         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],
 
         ...,
 
         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
  

Now we have to redefine the architecture and the step function of the A2C agent. We will use SpatialA2C as a proof of concept, but at the end it should inherit from the best version of A2C at our disposal.

Before we had to define 3 parameters that depended both on the model and on the state representation:
1. in_channels : screen + minimap channels
2. n_channels : number of channels at the end of the FullyConvSpatial
3. n_features : number of features at the end of the FullyConvNonSpatial

Now ideally if we want full control we should specify:
1. in_channels : screen + minimap channels
2. n_channels : number of channels at the end of the FullyConvSpatial
3. in_player : Number of features of the player_info (state['player']) in input
4. out_player : Number of features of the player_info (state['player']) in output
5. cat_n_channels = n_channels + out_player

The way I dealt before with this was:
1. Expose only n_channels and n_features as mandatory parameters
2. Specify all other input and model parameters in the dictionaries of the 2 models

Also now we want a FullyConvSpatial that accepts 2 inputs (spatial_state and player_state) and returns a single spatial_features with n_channels.

The new network looks like this:

```python
class FullyConvPlayerAndSpatial(nn.Module):
    def __init__(self, in_channels, in_player, player_features, conv_channels=32):
        super(FullyConvPlayerAndSpatial, self).__init__()
        self.conv_net = FullyConvSpatial(in_channels, conv_channels)
        self.fc_net = nn.Sequential(
                                    nn.Linear(in_player, 64),
                                    nn.ReLU(),
                                    nn.Linear(64, player_features),
                                    nn.ReLU()
                                    )
        
    def forward(self, spatial_state, player_state):
        spatial_x = self.conv_net(spatial_state)
        player_x = self.fc_net(player_state)
        spatial_features = self._cat_player_to_spatial(player_x, spatial_x)
        
    def _cat_player_to_spatial(self, player_x, spatial_x):
        """ 
        Assume spatial_x of shape (B, conv_channels, res, res).
        Cast player_x from (B, player_features) to (B, player_features, res, res)
        Concatenate spatial_x with the broadcasted player_x along the channel dim.
        """
        res = spatial_x.shape[-1]
        player_x = player_x.reshape((player_x.shape[:2]+(1,1,)))
        player_x = player_x.repeat(1,1,res,res)
        spatial_features = torch.cat([spatial_x, player_x], dim=1)
        return spatial_features
```

In [27]:
# Updated version in AC_modules/BatchedA2C

class FullSpaceA2C(SpatialA2C): 
    def __init__(self, action_space, env, spatial_model, nonspatial_model, spatial_dict, nonspatial_dict, 
                 n_features, n_channels, gamma=0.99, action_dict=None, H=1e-2, n_steps=20, device='cpu'):
        self.gamma = gamma
        self.n_actions = action_space
        self.n_steps = n_steps
        self.H = H
        self.AC = SpatialActorCritic_v4(action_space, env, spatial_model, nonspatial_model, spatial_dict, 
                                     nonspatial_dict, n_features, n_channels, action_dict=action_dict)
        self.device = device 
        self.AC.to(self.device) 
        
    def step(self, state, action_mask):
        spatial_state = state['spatial']
        player_state = state['player']
        spatial_state = torch.from_numpy(spatial_state).float().to(self.device)
        player_state = torch.from_numpy(player_state).float().to(self.device)
        action_mask = torch.tensor(action_mask).to(self.device)
        
        log_probs, spatial_features, nonspatial_features = self.AC.pi(spatial_state, player_state, action_mask)
        entropy = self.compute_entropy(log_probs)
        probs = torch.exp(log_probs)
        a = Categorical(probs).sample()
        a = a.detach().cpu().numpy()
        log_prob = log_probs[range(len(a)), a]
        
        args, args_log_prob, args_entropy = self.get_arguments(spatial_features, nonspatial_features, a)
        log_prob = log_prob + args_log_prob
        # Use only entropy of main actions for regularization
        #entropy = entropy + args_entropy

        action_id = np.array([self.AC.action_dict[act] for act in a])
        action = [actions.FunctionCall(action_id[i], args[i]) for i in range(len(action_id))]

        return action, log_prob, torch.mean(entropy)
    
    def compute_ac_loss(self, rewards, log_probs, entropies, states, done, bootstrap, trg_states): 
        """
        Notes:
        - Convention for tensors: batch first
        """
        
        # from list of dictionaries of arrays to 2 separate arrays
        spatial_states_lst = [s['spatial'] for s in states] #[(batch, other dims) x traj_len times]
        player_states_lst = [s['player'] for s in states] 
        spatial_states = torch.tensor(spatial_states_lst).float().to(self.device).transpose(1,0)
        player_states = torch.tensor(player_states_lst).float().to(self.device).transpose(1,0)
        
        spatial_states_lst_trg = [s['spatial'] for s in trg_states]
        player_states_lst_trg = [s['player'] for s in trg_states]
        spatial_states_trg = torch.tensor(spatial_states_lst_trg).float().to(self.device).transpose(1,0)
        player_states_trg = torch.tensor(player_states_lst_trg).float().to(self.device).transpose(1,0)
        
        # merge batch and episode dimensions
        old_spatial_states = spatial_states.reshape((-1,)+spatial_states.shape[2:])
        old_player_states = player_states.reshape((-1,)+player_states.shape[2:])
        
        average_n_steps = False # TRY ME
        if average_n_steps:
            # Use as V target the mean of 1-step to n-step V targets
            V_trg = []
            for n in range(1, self.n_steps + 1):
                n_step_V_trg = self.compute_n_step_V_trg(n, rewards, done, bootstrap, 
                                                         spatial_states_trg, player_states_trg)
                V_trg.append(n_step_V_trg)
            V_trg = torch.mean(torch.stack(V_trg, axis=0), axis=0)
        else:
            V_trg = self.compute_n_step_V_trg(self.n_steps, rewards, done, bootstrap, 
                                                         spatial_states_trg, player_states_trg)
            
        ### Wrap variables into tensors - merge batch and episode dimensions ###    
        log_probs = torch.stack(log_probs).to(self.device).transpose(1,0).reshape(-1)
        entropies = torch.stack(entropies, axis=0).to(self.device).reshape(-1)
        
        ### Compute critic and actor losses ###
        critic_loss = self.compute_critic_loss(old_spatial_states, old_player_states, V_trg)
        actor_loss, entropy = self.compute_actor_loss(log_probs, entropies, old_spatial_states, old_player_states, V_trg)

        return critic_loss, actor_loss, entropy
    
    def compute_n_step_V_trg(self, n_steps, rewards, done, bootstrap, spatial_states, player_states):
        """
        Compute m-steps value target, with m = min(n, steps-to-episode-end).
        Formula (for precisely n-steps):
            V^{(n)}(t) = \sum_{k=0}^{n-1} gamma^k r_{t+k} + gamma^n * V(s_t)
        """
        n_step_rewards, episode_mask, n_steps_mask_b = self.compute_n_step_rewards(rewards, done, n_steps)
        done[bootstrap] = False 
        # Check those
        new_spatial_states, Gamma_V, done = self.compute_n_step_states(spatial_states, done,
                                                                       episode_mask, n_steps_mask_b)
        new_player_states, _, _ = self.compute_n_step_states(player_states, done,
                                                                       episode_mask, n_steps_mask_b)
        
        new_spatial_states = new_spatial_states.reshape((-1,)+spatial_states.shape[2:])
        new_player_states = new_player_states.reshape((-1,)+player_states.shape[2:])
        
        done = torch.LongTensor(done.astype(int)).to(self.device).reshape(-1)
        n_step_rewards = torch.tensor(n_step_rewards).float().to(self.device).reshape(-1)
        Gamma_V = torch.tensor(Gamma_V).float().to(self.device).reshape(-1)
        
        with torch.no_grad():
            V_pred = self.AC.V_critic(new_spatial_states, new_player_states).squeeze()
            V_trg = (1-done)*Gamma_V*V_pred + n_step_rewards
            V_trg = V_trg.squeeze()
        return V_trg
    
    def compute_critic_loss(self, old_spatial_states, old_player_states, V_trg):
        V = self.AC.V_critic(old_spatial_states, old_player_states).squeeze()
        loss = F.mse_loss(V, V_trg)
        return loss
    
    def compute_actor_loss(self, log_probs, entropies, old_spatial_states, old_player_states, V_trg):
        with torch.no_grad():
            V_pred = self.AC.V_critic(old_spatial_states, old_player_states).squeeze()
        A = V_trg - V_pred
        policy_gradient = - log_probs*A
        policy_grad = torch.mean(policy_gradient)
        entropy = torch.mean(entropies)
        loss = policy_grad + self.H*entropy
        return loss, entropy

In [18]:
states = [state for _ in range(5)]

In [24]:
spatial_states_lst = [s['spatial'] for s in states]
player_states_lst = [s['player'] for s in states]
spatial_states_lst

[array([[[[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],
 
         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 1., 1., ..., 1., 1., 0.],
          [0., 1., 1., ..., 1., 1., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],
 
         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],
 
         ...,
 
         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0.

In [25]:
spatial_states = torch.tensor(spatial_states_lst)
player_states = torch.tensor(player_states_lst)
print(spatial_states.shape)
print(player_states.shape)

torch.Size([5, 2, 38, 32, 32])
torch.Size([5, 2, 8])


In [26]:
old_spatial_states = spatial_states.reshape((-1,)+spatial_states.shape[2:])
old_player_states = player_states.reshape((-1,)+player_states.shape[2:])
print(old_spatial_states.shape)
print(old_player_states.shape)

torch.Size([10, 38, 32, 32])
torch.Size([10, 8])


In addition to this we need to 
1. Store properly the 2 parts of the state (probably a list of dictionaries would do) - **DONE**
2. Obtain the tensors spatial_state and player_state with the "time" dimension too - **DONE**
3. Transpose them so the batch dimension is first - **DONE**
4. Finish to implement all changes in the update - **DONE**
5. Fix test and inspection_test - **DONE**
6. Fix all the inspections (storage, inspection_step and plots) - **TODO**, set inspection=False at the moment <br>
    i. Inspector compatible only with version2 (unique argument names)

# Sample all parameters in parallel

Idea: instead of having a ModuleDict with all networks separated for the different parameters, create a single network that outputs a (b, n_arguments, res, res) matrix of logits and then from there sample (b, n_arguments) arguments and finally access the ones that belong to the sampled actions.

This requires a fast parallel access to the arguments of each action, without looping on a dictionary.
To solve this, the idea is to use actions and parameters' identifiers and implement some kind of access like that (problem is, one action might have to access 2 or more parameters, so no 1 on 1).

In [16]:
# old forward pass
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical

def unravel_index(index, shape):
    out = []
    for dim in reversed(shape):
        out.append(index % dim)
        index = index // dim
    return tuple(reversed(out))

class SpatialParameters(nn.Module):
    
    def __init__(self, n_channels, linear_size):
        super(SpatialParameters, self).__init__()
        
        self.size = linear_size
        self.conv = nn.Sequential(nn.Conv2d(n_channels, 1, kernel_size=3, stride=1, padding=1))
    
    def forward(self, x, x_first=True):
        B = x.shape[0]
        x = self.conv(x)
        x = x.reshape((x.shape[0],-1))
        log_probs = F.log_softmax(x, dim=(-1))
        probs = torch.exp(log_probs)
        index = Categorical(probs).sample()
        print("index.shape: ", index.shape)
        y, x = unravel_index(index, (self.size,self.size))
        print("y.shape: ", y.shape)
        print("x.shape: ", x.shape)
        if x_first:
            arg_lst = [[xi.item(),yi.item()] for xi, yi in zip(x,y)]
        else:
            arg_lst = [[yi.item(),xi.item()] for xi, yi in zip(x,y)]
        log_prob = log_probs[torch.arange(B), index]
        return arg_lst, log_prob, log_probs

In [21]:
B = 3
n_channels = 16
res = 32
x = torch.rand(B,n_channels,res,res)

spatial_net = SpatialParameters(n_channels, res)
args, log_prob, log_probs = spatial_net(x)
print('args: ', args) # list of lists
print('log_prob: ', log_prob)
print('log_probs.shape: ', log_probs.shape) # (B, res**2)

index.shape:  torch.Size([3])
y.shape:  torch.Size([3])
x.shape:  torch.Size([3])
args:  [[19, 21], [8, 24], [0, 3]]
log_prob:  tensor([-6.8557, -7.0188, -6.6194], grad_fn=<IndexBackward>)
log_probs.shape:  torch.Size([3, 1024])


In [69]:
class ParallelSpatialParameters(nn.Module):
    
    def __init__(self, n_channels, linear_size, n_arguments):
        super(ParallelSpatialParameters, self).__init__()
        
        self.size = linear_size
        self.n_args = n_arguments
        self.conv = nn.Sequential(nn.Conv2d(n_channels, n_arguments, kernel_size=3, stride=1, padding=1))
    
    def forward(self, x, x_first=True):
        B = x.shape[0]
        x = self.conv(x)
        x = x.reshape((x.shape[0],self.n_args,-1))
        log_probs = F.log_softmax(x, dim=(-1))
        print('log_probs.shape: ', log_probs.shape)
        probs = torch.exp(log_probs)
        index = Categorical(probs).sample()
        print("index: ", index.shape, index)
        y, x = unravel_index(index, (self.size,self.size)) # shape (B,n_args)
        print("y: ", y.shape, y)
        print("x: ", x.shape, x)
        if x_first:
            arg_lst = np.array([[xi.detach().numpy(),yi.detach().numpy()] for xi, yi in zip(x,y)])
        else:
            arg_lst = np.array([[yi.detach().numpy(),xi.detach().numpy()] for xi, yi in zip(x,y)])
        arg_lst = arg_lst.transpose(0,2,1)  #shape (batch, n_arguments, [y,x]) (or [x,y])                 
        log_prob = log_probs.reshape(B*n_arguments, res**2)[torch.arange(B*n_arguments), index.reshape(-1)]
        log_prob = log_prob.reshape(B, n_arguments) 
        return arg_lst, log_prob, log_probs

In [70]:
B = 3
n_channels = 16
res = 32
n_arguments = 5
x = torch.rand(B,n_channels,res,res)

spatial_net = ParallelSpatialParameters(n_channels, res, n_arguments)
args, log_prob, log_probs = spatial_net(x)
print('args: ', args.shape, '\n', args) # list of lists
print('log_prob: ', log_prob)
print('log_probs.shape: ', log_probs.shape) # (B, res**2)

log_probs.shape:  torch.Size([3, 5, 1024])
index:  torch.Size([3, 5]) tensor([[  89,  480,   61,  741,  194],
        [ 284,  123,  367,  626,  825],
        [1016,   28,  331,  876,    3]])
y:  torch.Size([3, 5]) tensor([[ 2, 15,  1, 23,  6],
        [ 8,  3, 11, 19, 25],
        [31,  0, 10, 27,  0]])
x:  torch.Size([3, 5]) tensor([[25,  0, 29,  5,  2],
        [28, 27, 15, 18, 25],
        [24, 28, 11, 12,  3]])
args:  (3, 5, 2) 
 [[[25  2]
  [ 0 15]
  [29  1]
  [ 5 23]
  [ 2  6]]

 [[28  8]
  [27  3]
  [15 11]
  [18 19]
  [25 25]]

 [[24 31]
  [28  0]
  [11 10]
  [12 27]
  [ 3  0]]]
log_prob:  tensor([[-7.1027, -6.7878, -6.8794, -7.2924, -6.9437],
        [-7.1014, -6.9527, -6.9066, -7.1871, -7.0429],
        [-6.6213, -6.8631, -7.2751, -7.0676, -6.9554]], grad_fn=<ViewBackward>)
log_probs.shape:  torch.Size([3, 5, 1024])


In [64]:
# select a batch of specific arguments
args_indexes = torch.tensor([0,3,1]) # they should range between 0 and n_arguments - 1
args_selected = args[torch.arange(B), args_indexes]
args_selected #(batch, [y, x])

array([[ 1,  4],
       [31, 22],
       [16,  6]])

In [71]:
from pysc2.lib import actions 
def get_action_dict(action_names):
    action_ids = [actions.FUNCTIONS[a_name].id for a_name in action_names]
    action_dict = {i:action_ids[i] for i in range(len(action_ids))}
    return action_dict

In [74]:
action_names = ['select_army','select_rect','Move_screen']
action_dict = get_action_dict(action_names)

In [75]:
action_dict

{0: <_Functions.select_army: 7>,
 1: <_Functions.select_rect: 3>,
 2: <_Functions.Move_screen: 331>}

In [81]:
for a in action_dict:
    print("argumnets ids: ", action_dict[a] )

argumnets ids:  _Functions.select_army
argumnets ids:  _Functions.select_rect
argumnets ids:  _Functions.Move_screen


In this example I should get index from the sampling, because log_probs = indexes

In [32]:
index = torch.tensor([[240, 453, 154, 782, 191],
                        [160, 598, 538, 374, 873],
                        [924, 143, 775, 934, 685]])

y = torch.tensor([[ 7, 14,  4, 24,  5],
        [ 5, 18, 16, 11, 27],
        [28,  4, 24, 29, 21]])

x = torch.tensor([[16,  5, 26, 14, 31],
        [ 0, 22, 26, 22,  9],
        [28, 15,  7,  6, 13]])

In [44]:
log_probs = torch.arange(1024)
log_probs = log_probs.repeat(B, n_arguments, 1)
log_probs.shape

torch.Size([3, 5, 1024])

In [45]:
# this works
log_prob = log_probs.reshape(B*n_arguments, res**2)[torch.arange(B*n_arguments), index.reshape(-1)]

In [65]:
log_prob = log_prob.reshape(B, n_arguments)
log_prob.shape

torch.Size([3, 5])

In [47]:
torch.all(log_prob==index)

tensor(True)

# Using all possible actions and then masking out the ones not available

**How it works now:**
1. Define action names, e.g. 
```python 
action_names = ['select_army', 'Move_screen','Attack_screen'] 
```
2. Get the corresponding action dictionary, e.g.

```python 
from pysc2.lib import actions 

def get_action_dict(action_names):
    action_ids = [actions.FUNCTIONS[a_name].id for a_name in action_names]
    action_dict = {i:action_ids[i] for i in range(len(action_ids))}
    return action_dict

action_dict = get_action_dict(action_names)
action_dict
> {0:7,1:231,2:12}
```
3. Define action space

```python 
action_space = len(action_dict)
```

4. Pass both action space and action dict to SpatialActorCritic and use them like this

```python 
class SharedActor(nn.Module):
    def __init__(self, action_space, n_features):
        super(SharedActor, self).__init__()
        self.linear = nn.Linear(n_features, action_space)
        
# in SpatialActorCritic
def _init_params_nets(self):
        arguments_networks = {}
        self.arguments_dict = {}
        self.arguments_type = {}
        self.act_to_arg_names = {}
        
        for a in self.action_dict:
            action = self.all_actions[self.action_dict[a]]
            args = action.args
            self.act_to_arg_names[a] = [arg.name for arg in args]
            for arg in args:
                ### For clarity, check if argument has already been stored ###
                if arg.name in self.arguments_dict.keys():
                    continue
                else:
                    self.arguments_dict[arg.name] = arg.id # store 'name':id pairs for future use
                    if debug: print('\narg.name: ', arg.name)

                    size = self.all_arguments[arg.id].sizes
                    if debug: print('size: ', size)
                    if len(size) == 1:
                        if debug: 
                            print("Init CategoricalNet for "+arg.name+' argument')
                        arguments_networks[arg.name] = CategoricalNet(self.n_features, size[0]) 
                        self.arguments_type[arg.name] = 'categorical'
                    else:
                        if debug: print("Init SpatialNet for "+arg.name+' argument')
                        arguments_networks[arg.name] = SpatialParameters(self.n_channels, size[0]) 
                        self.arguments_type[arg.name] = 'spatial'
                    
        self.arguments_networks = nn.ModuleDict(arguments_networks)
```
5. Pass action dict to the function train_batched_A2C, where after some other passages will be used as follows

```python 
def get_action_mask(available_actions, action_dict):
    action_mask = ~np.array([action_dict[i] in available_actions for i in action_dict.keys()])
    return action_mask

available_actions = obs[0].observation.available_actions
action_mask = get_action_mask(available_actions, action_dict)
```
6. During the A2C step, we have to map `a` to `action_id`

```python
action_id = np.array([self.AC.action_dict[act] for act in a])
action = [actions.FunctionCall(action_id[i], args[i]) for i in range(len(action_id))]
```

7. Finally the actions `a` are used to get the names of their parameters

```python
# results is a dictionary like this
# results[arg_name] = (arg_sampled, log_prob, entropy)

for i, a in enumerate(action):
    arg_names = self.AC.act_to_arg_names[a]
    values = list( map(results.get, arg_names) )
```

**Observations:**
1. As we can see at point 4, action_dict is firstly used to access all_actions, from where we can get the arguments required by a specific action and consequently define the argument networks associated. Basically here we could skip a manual action dict and create one from all_actions. During this process we are creating the following dictionaries:
    - self.act_to_arg_names[a] - used to get names of the parameters of an action and then access the argument_networks with them
    - self.arguments_type[arg_name] - used to know which input to give to the argument network (spatial_features or nonspatial_features)
    - self.arguments_dict[arg_name] - actually just a list of all the arguments names would be enough, the values of this dictionary are never used
    - self.arguments_networks[arg_name] - used to sample arguments like this
    ```python
def sample_param(self, state_rep, arg_name):
        """
        Takes as input spatial_features if it's a spatial parameter,
        nonspatial_features otherwise.
        """
        return self.arguments_networks[arg_name](state_rep)
    ```
2. Since we don't have a natural correspondence between the integer `a` given by SharedActor and the `action_id` that is connected to that, we need to use the `action_dict` to remember those correspondences. If we use all actions, we can simply use `a=action_id` and forget about that dictionary. This should simplify a lot of things. 
3. When we use `get_action_mask`, we have the `available_actions` that is a numpy array of integers that contains the sc2 identifiers of the available action; the action mask instead is a numpy array as long as the action space and contains ones in the positions of actions that are NOT available. Here again things should get simpler if we use the correspondence `a=action_id`.

### How to sample parameters

The other thing that we have to think about is how to sample parameters. At the moment the most performing version was using a different parameter network for the same parameter when used by different actions, e.g. `Move_screen/screen` and `Attack_screen/screen` are 2 different networks. The problem with that is that it doesn't scale well if we consider 500 or more actions, because we will get around 1000 different parameters networks instead of something in the order of 30. - **IMPLEMENT THIS**

The idea outlined in SpatialA2C_v3 seemed good: embed the sampled action and use it as the weights of the 1x1 convolutional network for the screen arguments. Thinking more in depth about that is actually the same thing with a different interface (basically we shift the parameters from the convolutions to the embedding, but they will always be `n_actions x n_channels`, so the first version is preferable if it works better). 

### Sampling everything and selecting vs sampling only what needed

In the previous version of A2C we sampled all possible parameters for every sample in the batch (so to use the vectorization on the batch dimension) and then selected just the parameters of the actions that we needed, with a for loop. Trying out the same with more than 500 actions is not sustainable, there is a slowdown in performance that is too high. Probably a for loop also in the sampling would be preferable.

Old code:
```python
def get_arguments(self, spatial_features, nonspatial_features, action):
        """
        Samples all possible arguments for each sample in the batch, then selects only those that
        apply to the selected actions and returns a list containing the list of arguments for every 
        sampled action, the logarithm of the probability of sampling those arguments and the entropy 
        of their distributions. If an action has more arguments the log probs and the entropies returned
        are the sum of all those of the single arguments.
        """
        ### Sample and store each argument with its log prob and entropy ###
        results = {}    
        for arg_name in self.AC.arguments_names_lst:
            if self.AC.arguments_type[arg_name] == 'categorical':
                arg_sampled, log_prob, log_probs = self.AC.sample_param(nonspatial_features, arg_name)
            elif self.AC.arguments_type[arg_name] == 'spatial':
                arg_sampled, log_prob, log_probs = self.AC.sample_param(spatial_features, arg_name)
            else:
                raise Exception("argument type for "+arg_name+" not understood")  
            entropy = self.compute_entropy(log_probs)
            results[arg_name] = (arg_sampled, log_prob, entropy)
           
        ### For every action get the list of arguments and their log prob and entropy ###
        args, args_log_prob, args_entropy = [], [], []
        for i, a in enumerate(action):
            # default return values if no argument is sampled (like if there was a single value obtained with p=1)
            arg = []
            arg_log_prob = torch.tensor([0]).float().to(self.device)
            entropies = torch.tensor([0]).float().to(self.device)
            
            arg_names = self.AC.act_to_arg_names[a]
            values = list( map(results.get, arg_names) )
            if len(values) != 0:
                for j in range(len(values)):
                    # j is looping on the tuples (arg, log_prob, ent)
                    # Second index is for accessing tuples items
                    # i is for the sample index inside the batch
                    arg.append(list(values[j][0][i]))
                    arg_log_prob = arg_log_prob + values[j][1][i] # sum log_probs
                    entropies = entropies + values[j][2][i] # sum entropies
            args.append(arg)
            args_log_prob.append(arg_log_prob) 
            args_entropy.append(entropies)
            
        args_log_prob = torch.stack(args_log_prob, axis=0).squeeze()
        args_entropy = torch.stack(args_entropy, axis=0).squeeze()
        return args, args_log_prob, args_entropy

```

So basically in this code we loop on `self.AC.arguments_names_lst`, find out if the argument is spatial or categorical, then sample the argument giving in input the proper features (spatial or not) with batch dimension first, computing the entropy term and storing everything in a dictionary accessible with the arg_name keyword.
In the second part of the function, for each action we get a list with the names of the arguments that it requires, collect the right index along the batch dimension and compute the total log prob and entropy (as the sum of all log probs and entropies respectively). Finally we batch together the lists obtained from the loop.

All of this was done in the belief that sampling everything while keeping the batch dimension was more efficient than looping along the batch dimension, but this is probably not true anymore, since the action space exploded (573 unique actions, 759 unique argument networks, no more than 36 samples per batch). With these proportions we can expect around 50 sequential loops considering the average number of parameters per action 759/573 = 1.3, which is much better than 759 loops.

So this time we start directly from the second part of the funciton and do the sampling in-place:
1. Loop on actions
2. Get arg_names
3. Loop on them
4. Discriminate between spatial and categorical
5. Slice the spatial or nonspatial features selecting the index i along the batch axis, then add again a fictious batch dimension with .unsqueeze(0)
6. Collect the results as done before
7. Stack list of results as before

In [15]:
def get_arguments(self, spatial_features, nonspatial_features, action):
    """
    1. Loop on actions
    2. Get arg_names
    3. Loop on them
    4. Discriminate between spatial and categorical
    5. Slice the spatial or nonspatial features selecting the index i along the batch axis, 
       then add again a fictious batch dimension with .unsqueeze(0)
    6. Collect the results 
    7. Stack list of results
    """
    ### For every action get the list of arguments and their log prob and entropy ###
    args, args_log_prob, args_entropy = [], [], []
    # 1)
    for i, a in enumerate(action):
        # default return values if no argument is sampled (like if there was a single value obtained with p=1)
        arg = []
        arg_log_prob = torch.tensor([0]).float().to(self.device)
        entropies = torch.tensor([0]).float().to(self.device)
        # 2)
        arg_names = self.AC.act_to_arg_names[a]
        if len(arg_names) != 0:
            # 3)
            for arg_name in arg_names:
                # 4)
                if self.AC.arguments_type[arg_name] == 'categorical':
                    # 5)
                    x = nonspatial_features[i].unsqueeze(0)
                    arg_sampled, log_prob, log_probs = self.AC.sample_param(x, arg_name)
                elif self.AC.arguments_type[arg_name] == 'spatial':
                    x = spatial_features[i].unsqueee(0)
                    arg_sampled, log_prob, log_probs = self.AC.sample_param(x, arg_name)
                else:
                    raise Exception("argument type for "+arg_name+" not understood")  
                entropy = self.compute_entropy(log_probs)

                # 6)
                arg.append(arg_sampled)
                arg_log_prob = arg_log_prob + log_prob 
                entropies = entropies + entropy
        args.append(arg)
        args_log_prob.append(arg_log_prob) 
        args_entropy.append(entropies)

    # 7)
    args_log_prob = torch.stack(args_log_prob, axis=0).squeeze()
    args_entropy = torch.stack(args_entropy, axis=0).squeeze()
    return args, args_log_prob, args_entropy

### Inspection

How it worked:
1. Define action dict
2. Create a dictionary with all unique arguments associated to the action dict, key=arg_name+"_distr", value = []
3. For each argument we also save if it's spatial or not

TODO:
1. Support dictionary states {"spatial","player"} - should be automatic
2. Use the agent to init the Inspector, so we can simply copy all the info that we need from agent.AC


In [9]:
agent.AC

FullSpatialActorCritic(
  (spatial_features_net): FullyConvPlayerAndSpatial(
    (conv_net): FullyConvSpatial(
      (net): Sequential(
        (0): Conv2d(36, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
        (1): ReLU()
        (2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (3): ReLU()
      )
    )
    (fc_net): Sequential(
      (0): Linear(in_features=8, out_features=64, bias=True)
      (1): ReLU()
      (2): Linear(in_features=64, out_features=16, bias=True)
      (3): ReLU()
    )
  )
  (nonspatial_features_net): FullyConvNonSpatial(
    (conv): Sequential(
      (0): Conv2d(48, 64, kernel_size=(3, 3), stride=(2, 2))
      (1): ReLU()
    )
    (net): Sequential(
      (0): Linear(in_features=14400, out_features=256, bias=True)
      (1): ReLU()
    )
  )
  (actor): SharedActor(
    (linear): Linear(in_features=256, out_features=573, bias=True)
  )
  (critic): SharedCritic(
    (net): Linear(in_features=256, out_features=1, bias=True

In [10]:
import torch.nn.functional as F 

In [15]:
p = F.softmax(torch.rand(10), dim=-1)

In [43]:
# get 5 most probable actions
idx = np.argsort(p.numpy())
top_5 = idx[-5:]
top_5 = np.array(top_5[::-1]) # some issues in accessing p if I don't call np.array()
top_5

array([5, 8, 9, 1, 7])

In [40]:
p[top_5]

tensor([0.1527, 0.1241, 0.1140, 0.1122, 0.1029])